In [ ]:
!pip install geneticalgorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from collections import defaultdict
from geneticalgorithm import geneticalgorithm as ga

In [ ]:
food_type_dict = {

    'main':
        {
            'price': 50,
            'please_level': 30,
            'fresh_rate': [0.5, 1.0],
            'bug_rate': 0.05,
            'bug_affective_rate': 0.05,
            'overtake_plus_rate': 1.8
        },

    'side':
        {
            'price': 20,
            'please_level': 15,
            'fresh_rate': [0.5, 1.0],
            'bug_rate': 0.05,
            'bug_affective_rate': 0.05,
            'overtake_plus_rate': 1.8
        },

    'little':
        {
            'price': 10,
            'please_level': 10,
            'fresh_rate': [0.5, 1.0],
            'bug_rate': 0.05,
            'bug_affective_rate': 0.05,
            'overtake_plus_rate': 1.8
        }
}


class food:

    def __init__(self, food_type, overtake):
        self.info = food_type_dict[food_type]
        self.overtake = overtake

    def get_satisfactory(self):
        _fresh_rate = np.random.uniform(
            self.info['fresh_rate'][0], self.info['fresh_rate'][1])
        _is_bug = bool(np.random.binomial(1, self.info['bug_rate'], 1) == 1)

        if _is_bug:
            satisfactory = _fresh_rate * \
                self.info['please_level']*self.info['bug_affective_rate']
        else:
            satisfactory = _fresh_rate*self.info['please_level']

        if self.overtake:
            satisfactory *= self.info['overtake_plus_rate']
        return satisfactory

    def get_food_price(self):
        return self.info['price']

In [ ]:
crit_type_dict = {
    'boss': [0.8, [5, 10, 15], [.8, .15, .05]],
    'boss_wife': [0.6, [5, 10], [.9, .1]],
    'granny': [0.4, [5], [1.0]]
}


class elite_employee:

    def __init__(self, crit_type):
        self.crit_type = crit_type_dict[crit_type]

    def challenge_the_price(self, food_price):
        uAreToOver = bool(np.random.binomial(1, self.crit_type[0], 1) == 1)
        if uAreToOver:
            crit_damage = np.random.choice(
                self.crit_type[1], 1, p=self.crit_type[2])[0]
            return food_price+crit_damage
        else:
            return food_price

    def cal_end_price(self, food_obj):
        if food_obj.overtake:
            food_end_price = self.challenge_the_price(
                food_obj.get_food_price())
            return food_end_price
        return food_obj.get_food_price()

In [ ]:
strategy_x1_dict = {
    'normal': [food('main', False), food('side', False), food('little', False)],
    'combo': [food('main', False), food('side', True), food('little', True)],
    'side': [food('main', False), food('side', True), food('little', False)],
    'little': [food('main', False), food('side', False), food('little', True)],
}

#main, side, little
strategy_x2_dict = {
    '1main2side2little': [1, 2, 2],
    '1main1side3little': [1, 1, 3],
    '1main3little': [1, 0, 3],
    '2main2little': [0, 2, 2],
    '2side3little': [0, 2, 3],
    'forGA': [],
}


class customer:

    def __init__(self, strategy_x1, strategy_x2, employee_obj=None, penalty_rate=2000):
        self.food_obj_list = strategy_x1_dict[strategy_x1]
        self.stra2 = strategy_x2_dict[strategy_x2]
        self.food_price_list = None
        self.opponent = employee_obj
        self.penalty_rate = penalty_rate
        self.price_buffer = []

    def get_total_satisfactory(self):
        _m = [self.food_obj_list[0].get_satisfactory()
              for _ in range(self.stra2[0])]
        _s = [self.food_obj_list[1].get_satisfactory()
              for _ in range(self.stra2[1])]
        _l = [self.food_obj_list[2].get_satisfactory()
              for _ in range(self.stra2[2])]

        return sum(_m+_s+_l)

    def get_total_satisfactory_GA(self, numlist):
        _m = [self.food_obj_list[0].get_satisfactory()
              for _ in range(int(numlist[0]))]
        _s = [self.food_obj_list[1].get_satisfactory()
              for _ in range(int(numlist[1]))]
        _l = [self.food_obj_list[2].get_satisfactory()
              for _ in range(int(numlist[2]))]

        buffer = 0
        idx = 0
        for n in numlist:
            n = int(n)
            for _ in range(n):
                afterprice = self.opponent.cal_end_price(
                    self.food_obj_list[idx])
                buffer += afterprice
            idx += 1
        self.price_buffer.append(buffer)
        if buffer > 130:
            return -(sum(_m+_s+_l)-self.penalty_rate*buffer)

        return -(sum(_m+_s+_l))

    def init_food_price(self):
        _m = [self.food_obj_list[0].get_food_price()
              for _ in range(self.stra2[0])]
        _s = [self.food_obj_list[1].get_food_price()
              for _ in range(self.stra2[1])]
        _l = [self.food_obj_list[2].get_food_price()
              for _ in range(self.stra2[2])]

        self.food_price_list = _m+_s+_l

    def after_challenge_price(self, elite_emp_obj):
        buffer = 0
        idx = 0

        for n in self.stra2:
            for _ in range(n):
                afterprice = elite_emp_obj.cal_end_price(
                    self.food_obj_list[idx])
                buffer += afterprice
            idx += 1
        return buffer

In [ ]:
############# Monte Carlo Simulation ########################

S = int(input('請輸入要模擬的次數'))
types1 = ['normal', 'combo', 'side', 'little']
types2 = ['1main2side2little', '1main1side3little',
          '1main3little', '2main2little', '2side3little']
employees = ['boss', 'boss_wife', 'granny']

result_dict = defaultdict()

for t1 in types1:
    result_dict[t1] = {}
    for t2 in types2:
        result_dict[t1][t2] = {}
        for e in employees:
            result_dict[t1][t2][e] = {}
            employee = elite_employee(e)
            cus = customer(t1, t2, employee_obj=employee)
            

            satifactory_list = []
            end_price_list = []

            for _ in range(S):
                satifactory_list.append(cus.get_total_satisfactory())
                end_price_list.append(cus.after_challenge_price(employee))

            result_dict[t1][t2][e]['avg_satisfactory'] = np.mean(
                satifactory_list)
            result_dict[t1][t2][e]['avg_price'] = np.mean(end_price_list)
            print(
                f"{t1} {t2} {e} \navg_satisfactory: {result_dict[t1][t2][e]['avg_satisfactory']} || avg_price: {result_dict[t1][t2][e]['avg_price']}\n\n")

#             plt.subplot(221)
#             plt.hist(satifactory_list)
#             plt.subplot(222)
#             plt.hist(end_price_list)
#             plt.title(t1+' '+ t2+ ' ' + e)
#             plt.show()

In [ ]:
############# Genetic Algo ########################

types1 = ['normal', 'combo', 'side', 'little']
employees = ['boss', 'boss_wife', 'granny']

result_dict = defaultdict()
varbound = np.array([[1, 7]]*3)

for t1 in types1:
    for e in employees:
        print('\n')
        print(t1,'\t', e)
        employee = elite_employee(e)
        cus = customer(t1, 'forGA', employee_obj=employee)
        
        model = ga(function=cus.get_total_satisfactory_GA, dimension=3,
                   variable_type='real', variable_boundaries=varbound)

        model.run()
        print('Average Price: ',sum(cus.price_buffer)/len(cus.price_buffer))
        print('\n')
        print('='*80,'\n')